# Estratégia Quantitativa de Pairs Trading com Dados da B3

Este notebook implementa uma estratégia quantitativa de pairs trading com ações da B3. O pipeline inclui:

- Carregamento de dados históricos de preços e volumes de ações usando banco SQLite e Yahoo Finance.
- Seleção de pares cointegrados via teste estatístico (ADF e KPSS).
- Estratégia baseada em bandas de volatilidade com z-score.
- Backtest da estratégia em períodos sequenciais.
- Análise dos resultados e métricas de performance.

Autor: Rafael Eiki Teruya, aluno de Ciências Econômicas na FEA-USP

# 📊 Backtest de Pairs Trading
---

## 📝 Descrição

Este notebook implementa um **pipeline completo de Pairs Trading** utilizando dados da B3 armazenados em **banco SQLite**.  
O fluxo inclui:

1️⃣ Criação e manutenção do banco de dados de ativos (`ativos_b3.db`)  
2️⃣ Download e armazenamento de dados históricos via `yfinance`  
3️⃣ Carregamento de tickers com filtro de **volume mínimo**  
4️⃣ Seleção de pares cointegrados com testes ADF e KPSS  
5️⃣ Cálculo do spread e z-score exponencial (EWM)  
6️⃣ Geração de sinais de entrada/saída  
7️⃣ Backtest diário com portfolio ponderado por volatilidade  
8️⃣ Validação walk-forward para reduzir overfitting  

---

## ⚙️ Objetivos

- Manter um **banco de dados atualizado** com preços, volume e retornos da B3  
- Selecionar pares de ativos com **cointegração estável e meia-vida adequada**  
- Testar estratégias de **long-short baseadas em z-score**  
- Avaliar **performance do portfólio** (Retorno, Sharpe ajustado pelo CDI)  
- Garantir **validação fora da amostra** usando walk-forward  

---

## 📌 Estrutura do Notebook

1️⃣ 🏗️ Criação do banco de dados SQLite e tabelas  
2️⃣ 💾 Download de preços e salvamento no banco (`yfinance`)  
3️⃣ 📂 Carregamento de tickers e filtragem por volume  
4️⃣ 🔍 Seleção de pares cointegrados (ADF + KPSS)  
5️⃣ 📊 Cálculo do spread e z-score EWM  
6️⃣ ⚡ Geração de sinais de entrada e saída  
7️⃣ 💹 Backtest diário com portfolio ponderado  
8️⃣ 📈 Métricas de performance e análise walk-forward  



## Carregando Bibliotecas e configurações iniciais

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller, kpss
from scipy.stats import pearsonr
import itertools
import vectorbt as vbt
import warnings
import sqlite3
import cvxpy as cp
from datetime import datetime
import fundamentus as fmt
import logging
from itertools import product
import talib as ta
from hurst import compute_Hc
# Configurações
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

2025-09-03 17:17:48,333 [logging.log_init] INFO: LOGLEVEL=INFO


In [1]:
# Configuração do backtest
inicio_total = '2017-01-01' 
fim_total = '2024-12-31'   
janela_otimizacao = '3Y' 
periodo_teste = '1Y'

##  Criação do banco de dados
# Banco de Dados de Ações da B3

Nessa etapa criamos e alimentamos um banco de dados **SQLite** com informações históricas de ações da **B3**, utilizando a biblioteca **yfinance** para coleta de dados.

---

## Estrutura do Banco de Dados
- **Nome do banco:** `dados_b3.db`  
- **Tabela:** `acoes`  
- **Colunas:**
  - `data` (TEXT, NOT NULL) → Data da observação
  - `ticker` (TEXT, NOT NULL) → Código da ação
  - `preco` (REAL) → Preço de fechamento
  - `volume` (REAL) → Volume negociado
  - `retorno` (REAL) → Retorno diário (variação percentual)
  - **Chave primária:** `(data, ticker)`


In [5]:
# Nome do database
DB_NAME = 'ativos_b3.db'

# Criando a tabela
def criar_tabela():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS acoes (
            data TEXT NOT NULL,
            ticker TEXT NOT NULL,
            preco REAL,
            volume REAL,
            retorno REAL,
            PRIMARY KEY (data, ticker)
        )
    """)
    conn.commit()
    conn.close()

# Função para salvar um ticker
def salvar_ticker(ticker, inicio, fim):
    df = yf.download(ticker, start=inicio, end=fim, progress=False)
    df_empty = []
    if df.empty:
        df_empty.append(ticker)
        print(f"⚠️ Alerta: dados vazios para {df_empty}")

    df['retorno'] = df['Close'].pct_change()
    df = df.reset_index()
    df = df[['Date', 'Close', 'Volume', 'retorno']]
    df.columns = ['data', 'preco', 'volume', 'retorno']
    df['ticker'] = ticker

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    for _, row in df.iterrows():
        cursor.execute("""
            INSERT OR REPLACE INTO acoes (data, ticker, preco, volume, retorno)
            VALUES (?, ?, ?, ?, ?)
        """, (row['data'].strftime('%Y-%m-%d'), row['ticker'],
              row['preco'], row['volume'], row['retorno']))
    conn.commit()
    conn.close()

# Executando para todos os tickers da B3
if __name__ == "__main__":
    criar_tabela()

    tickers = []
    for setor in range(44):
        try:
            tickers.extend(fmt.list_papel_setor(setor))
        except:
            continue
    tickers = [s + ".SA" for s in tickers if s]
    tickers = tickers + ["^BVSP"]

    start_date = '2017-01-01'
    end_date = '2025-01-01'

    for t in tickers:
        salvar_ticker(t, start_date, end_date)
    print(f"✅ {len(tickers)-13} ativos salvos no banco")

⚠️ Alerta: dados vazios para ['IGBR3.SA']
⚠️ Alerta: dados vazios para ['JPSA3.SA']
⚠️ Alerta: dados vazios para ['ADMF3.SA']
⚠️ Alerta: dados vazios para ['ELMD3.SA']
⚠️ Alerta: dados vazios para ['BLUT3.SA']
⚠️ Alerta: dados vazios para ['BLUT4.SA']
⚠️ Alerta: dados vazios para ['AZTE3.SA']
⚠️ Alerta: dados vazios para ['NTCO3.SA']
⚠️ Alerta: dados vazios para ['TOKY3.SA']
⚠️ Alerta: dados vazios para ['ATMP3.SA']
⚠️ Alerta: dados vazios para ['CTAX3.SA']
⚠️ Alerta: dados vazios para ['GOLL4.SA']
⚠️ Alerta: dados vazios para ['MOTV3.SA']
⚠️ Alerta: dados vazios para ['RVEE3.SA']
✅ 408 ativos salvos no banco


## Funções para Carregar Dados do Banco de Dados

Estas funções facilitam o acesso aos dados salvos no banco SQLite:

- `carregar_ticker`: Busca dados históricos de um ativo específico, filtrando por ticker e período. Retorna DataFrame indexado por data.
- `carregar_tickers`: Carrega vários ativos simultaneamente, filtrando por volume médio mínimo para garantir liquidez. Retorna DataFrame com cada ativo em uma coluna.

Essas funções são usadas para preparar os dados para seleção de pares e backtest.

In [6]:
def carregar_ticker(ticker, start=None, end=None):
    conn = sqlite3.connect(DB_NAME)

    query = f"""
        SELECT data, preco, volume, retorno
        FROM acoes
        WHERE ticker = ?
        {f"AND data >= '{start}'" if start else ""}
        
        {f"AND data <= '{end}'" if end else ""}
        ORDER BY data
    """

    df = pd.read_sql(query, conn, params=(ticker,), parse_dates=["data"])
    conn.close()

    df.set_index("data", inplace=True)

    return df

def  carregar_tickers(tickers, start=None, end=None, start_test=None, end_test=None):
    dfs = []
    for ticker in tickers:
        df = carregar_ticker(ticker, start, end)
        volume_df = carregar_ticker(ticker, start_test, end_test)
        min_volume = 100_000
        if volume_df['volume'].mean() >= min_volume:
            df = df[['preco']].rename(columns={"preco": ticker})
            dfs.append(df)
    if dfs:
        dados = pd.concat(dfs, axis=1)
        dados.columns = [ticker for ticker in tickers if ticker in dados.columns]
        return dados, dados.columns
    else:
        return None

# Seleção de Pares Cointegrados

## 1️⃣ Transformação Logarítmica dos Preços
Para cada par de ativos $X_t$ e $Y_t$, usamos os **preços logarítmicos**:

$$
x_t = \ln(X_t), \quad y_t = \ln(Y_t)
$$

> Isso transforma retornos multiplicativos em aditivos e estabiliza a variância.

---

## 2️⃣ Estimação do Beta (Hedge Ratio)
O beta ($\beta$) relaciona $Y_t$ com $X_t$ via regressão linear:

$$
x_t = \alpha + \beta y_t + \epsilon_t
$$

- $\alpha$ = intercepto  
- $\beta$ = hedge ratio  
- $\epsilon_t$ = resíduos (spread)

O beta indica **quanto de $Y$ é necessário para cobrir $X$**.

---

## 3️⃣ Cálculo do Spread
O **spread estacionário** é definido como:

$$
S_t = x_t - \beta y_t
$$

- Se os ativos forem cointegrados, $S_t$ será **estacionário** (média constante, variação limitada).

---

## 4️⃣ Testes de Estacionariedade

### a) Teste ADF (Augmented Dickey-Fuller)
Verifica se o spread tem raiz unitária ($H_0$: não estacionário):

$$
\Delta S_t = \phi S_{t-1} + \sum_{i=1}^{p} \gamma_i \Delta S_{t-i} + \varepsilon_t
$$

- Rejeitar $H_0$ (p-valor < 0.01) indica **spread estacionário**.

### b) Teste KPSS
Verifica a hipótese nula inversa ($H_0$: estacionário).  
- Desejamos p-valor > 0.05 para confirmar estacionariedade.

---

## 5️⃣ Meia-Vida da Reversão à Média
Modelo AR(1) para o spread:

$$
\Delta S_t = \lambda S_{t-1} + \eta_t
$$

A **meia-vida** ($\tau$) é o tempo médio para o spread voltar à média:

$$
\tau = -\frac{\ln(2)}{\lambda}
$$

> Filtramos pares com $5 < \tau < 60$ dias para estratégias de curto prazo.

---

## 6️⃣ Volatilidade
A volatilidade anualizada do spread e dos ativos é calculada por:

$$
\sigma_\text{ativo} = \text{std}(\text{retornos diários}) \times \sqrt{252}
$$

$$
\sigma_\text{spread} = \text{std}(S_t) \times \sqrt{252}
$$

> Essencial para dimensionar posições e controlar risco no backtest.

---

## 7️⃣ Seleção Final de Pares
A função seleciona **os 3 pares mais promissores** com base nos critérios:

1. Spread **estacionário** (ADF < 0.01, KPSS > 0.05)  
2. **Meia-vida** entre 5 e 60 dias  
3. **Beta** entre 0.3 e 3  
4. Número suficiente de observações  
5. Retorna métricas de volatilidade para gestão de risco  

> Ao final, são escolhidos **exatamente 3 pares** para o portfólio, priorizando os spreads mais estacionários (menor p-valor do ADF).





In [35]:
def selecionar_pares(precos, n_acoes=3):
    pares_cointegrados = []
    tickers = precos.columns.tolist()
    ativos_usados_como_ativo1 = set()  # Conjunto para rastrear ativos já usados como Ativo1

    # Gerar todas as combinações possíveis de pares
    for ativo1, ativo2 in itertools.combinations(tickers, 2):
        if ativo1 in ativos_usados_como_ativo1:
            continue  # Pular se o ativo1 já foi usado

        df_par = precos[[ativo1, ativo2]].dropna()
        if len(df_par) < 100:  # Ignorar pares com poucos dados
            continue

        # Preços log dos ativos
        df_par_log = np.log(df_par)

        try:
            # Estimando o beta
            X = sm.add_constant(df_par_log[ativo2])
            y = df_par_log[ativo1].values
            regressao = sm.OLS(y, X)
            modelo = regressao.fit()
            beta = modelo.params[1]
        except:
            continue

        if np.abs(beta) < 1e-5:
            continue

        # Calcular spread
        spread = df_par_log[ativo1] - beta * df_par_log[ativo2]

        # Calcular p-valor do teste ADF
        try:
            p_valor = adfuller(spread)[1]
            p_valor_kpss = kpss(spread)[1]
        except:
            continue

        # Calcular meia vida
        spread_lag1 = spread.shift(1).dropna()
        spread_delta = spread - spread_lag1

        df_reg = pd.DataFrame({'spread_delta': spread_delta.iloc[1:], 'spread_lag1': spread_lag1})

        X = sm.add_constant(df_reg['spread_lag1'])
        modelo = sm.OLS(df_reg['spread_delta'], X).fit()
        meia_vida = -np.log(2) / modelo.params['spread_lag1']

        # Selecionar pares com spread estacionário
        if p_valor < 0.01 and 5 < meia_vida < 60 and 0.3 < np.abs(beta) < 3 and p_valor_kpss > 0.05:
            # Calcular vol dos retornos
            retorno_1 = df_par[ativo1].pct_change()
            retorno_2 = df_par[ativo2].pct_change()

            vol_retorno_1 = retorno_1.std()*np.sqrt(252)
            vol_retorno_2 = retorno_2.std()*np.sqrt(252)

            vol_spread = spread.std()*np.sqrt(252)

            pares_cointegrados.append({
                'Ativo1': ativo1,
                'Ativo2': ativo2,
                'Vol_ativo1': vol_retorno_1,
                'Vol_ativo2': vol_retorno_2,
                'Beta': beta,
                'spread': spread,
                'vol-spread': vol_spread,
                'p_valor_adf': p_valor,
                'meia_vida': meia_vida
            })

            # Adicionar ativo1 ao conjunto de ativos usados
            ativos_usados_como_ativo1.add(ativo1)

    # Retornar DataFrame com os pares selecionados
    return pd.DataFrame(pares_cointegrados).sort_values(by='p_valor_adf', ascending=True).iloc[0:n_acoes]

# Função de Backtest (Walk-Forward)

A função principal executa o walk-forward, aplicando o pipeline completo:

- Seleção de pares cointegrados para cada janela.
- Backtest no período seguinte.
- Consolidação dos resultados.

Permite avaliar a estratégia em diferentes períodos e validar sua robustez.

# Walk-Forward Backtest de Pares – Matemática

## 1️⃣ Seleção do Spread

Para cada par de ativos $(X, Y)$ no período de otimização:

$  
\text{Spread}_t = P_{X,t} - \beta \cdot P_{Y,t}  
$  

- $P_{X,t}$ e $P_{Y,t}$ são os preços ajustados dos ativos X e Y no tempo $t$.  
- $\beta$ é obtido pelo ajuste de regressão linear ou outro método de cointegração.

---

## 2️⃣ Média Móvel Exponencial (EWM)

Calculamos a média móvel exponencial e o desvio padrão exponencial do spread:

$  
\mu_t = \text{EWM}(\text{Spread}_t, \text{halflife}=h)  
$  

$  
\sigma_t = \text{EWM}(\text{Spread}_t, \text{halflife}=h, \text{std})  
$  

- `halflife` define o tempo necessário para que pesos exponenciais decaiam pela metade.  
- Incluímos o período de otimização + teste para inicializar a EWM.

---

## 3️⃣ Cálculo do Z-Score

$  
Z_t = \frac{\text{Spread}_t - \mu_t}{\sigma_t}  
$  

- Padroniza o spread para identificar extremos.  
- Usamos `shift(1)` para evitar **lookahead bias**.

---

## 4️⃣ Quantis para Sinais

Definimos quantis do z-score para entrada, saída, SL e TP:

$  
\begin{aligned}  
q_{\text{entrada long}} &= Q_{0.05}(Z) \\  
q_{\text{saida long}} &= Q_{0.25}(Z) \\  
q_{\text{entrada short}} &= Q_{0.95}(Z) \\  
q_{\text{saida short}} &= Q_{0.75}(Z) \\  
q_{\text{SL long}} &= Q_{0.005}(Z) \\  
q_{\text{TP long}} &= Q_{0.50}(Z) \\  
q_{\text{SL short}} &= Q_{0.995}(Z) \\  
q_{\text{TP short}} &= Q_{0.50}(Z)  
\end{aligned}  
$  

---

## 5️⃣ Geração de Sinais

- Entrada Long:

$  
\text{entrada\_long}_t = \mathbf{1}_{\{Z_t < q_{\text{entrada long}}\}}  
$  

- Saída Long:

$  
\text{saida\_long}_t = \mathbf{1}_{\{Z_t > q_{\text{saida long}} \text{ ou } Z_t < q_{\text{SL long}} \text{ ou } Z_t > q_{\text{TP long}}\}}  
$  

- Entrada Short:

$  
\text{entrada\_short}_t = \mathbf{1}_{\{Z_t > q_{\text{entrada short}}\}}  
$  

- Saída Short:

$  
\text{saida\_short}_t = \mathbf{1}_{\{Z_t < q_{\text{saida short}} \text{ ou } Z_t > q_{\text{SL short}} \text{ ou } Z_t < q_{\text{TP short}}\}}  
$  

- $\mathbf{1}_{\{\cdot\}}$ é a função indicadora.

---

## 6️⃣ Stops Baseados em Volatilidade

$  
\text{SL}_t = k_{\text{SL}} \cdot \sigma_{\text{spread}}  
$  

$  
\text{TP}_t = k_{\text{TP}} \cdot \sigma_{\text{spread}}  
$  

- $k_{\text{SL}} = 2$, $k_{\text{TP}} = 4$ (ajustáveis).  

---

## 7️⃣ Portfólio Ponderado por Volatilidade

Para $N$ pares:

$  
w_i = \frac{1 / \sigma_i}{\sum_{j=1}^{N} 1 / \sigma_j}  
$  

- $\sigma_i$ é a volatilidade do spread do par $i$.  
- Equity do portfólio:

$  
\text{Equity}_t = \sum_{i=1}^{N} w_i \cdot \text{Equity}_{i,t}  
$  

---

## 8️⃣ Métricas de Performance

- Retorno do período:

$  
R = \frac{\text{Equity}_{\text{final}}}{\text{Equity}_{\text{inicial}}} - 1  
$  

- Sharpe Ratio ajustado pelo CDI:

$  
\text{Sharpe} = \frac{\bar{r} - r_f}{\sigma_r} \cdot \sqrt{252}  
$  

- $\bar{r}$ = retorno diário médio do portfólio  
- $\sigma_r$ = desvio padrão diário  
- $r_f$ = CDI diário: $r_f = (1 + \text{CDI anual})^{1/252} - 1$



In [64]:
def run_walk_forward():
    resultados = []
    lista_portfolio = []
    datas_otimizacao = pd.date_range(start=inicio_total, end=fim_total, freq='AS')  # 'AS' = Ano Início

    janela_anos = 3
    for i in range(len(datas_otimizacao)):
        inicio_opt = datas_otimizacao[i]
        fim_opt = inicio_opt + pd.DateOffset(years=janela_anos) - pd.DateOffset(days=1)
        inicio_teste = fim_opt + pd.DateOffset(days=1)
        fim_teste = inicio_teste + pd.DateOffset(years=1) - pd.DateOffset(days=1)

        # Ignora períodos incompletos
        
        if fim_teste > pd.to_datetime(fim_total):
            continue  
        
        print(f"\n=== Período {i+1} ===")
        print(f"Otimização: {inicio_opt.date()} a {fim_opt.date()} (3 anos)")
        print(f"Teste: {inicio_teste.date()} a {fim_teste.date()} (1 ano)")
        
        
        # Passo 1: Selecionar pares no período de otimização
        try:
            # Carregar lista de ações da B3
            stocks = []
            for setor in range(44):
                try:
                    stocks.extend(fmt.list_papel_setor(setor))
                except:
                    continue
            stocks = [s + ".SA" for s in stocks if s]
            
            # Baixar dados e filtrar por volume
            dados_opt, tickers_filtrados = carregar_tickers(stocks, inicio_opt, fim_opt, inicio_teste,fim_teste)
            if dados_opt is None:
                continue
        
            precos_opt = dados_opt[tickers_filtrados].dropna(axis=1)

            df_pares = selecionar_pares(precos_opt)
            if df_pares.empty:
                print("Nenhum par cointegrado encontrado.")
                continue

            for i in range(len(df_pares)):

                ticker1, ticker2 = df_pares['Ativo1'].iloc[i], df_pares['Ativo2'].iloc[i]
                beta_otimo = df_pares['Beta'].iloc[i]

                dados_teste, _ = carregar_tickers([ticker1, ticker2], inicio_teste, fim_teste, inicio_teste,fim_teste)

            # Passo 2: Testar estratégia no período seguinte
                try:

                    preco1 = dados_teste[ticker1].dropna()
                    preco2 = dados_teste[ticker2].dropna()

                    precos_teste = pd.concat([preco1, preco2], axis=1, join='inner')
                    precos_teste.columns = [ticker1, ticker2]

                    if precos_teste.empty:
                        continue

                    # Selecionar beta
                    beta = df_pares['Beta'].iloc[i]
                    
                    # Spread do período de teste
                    spread_teste = precos_teste[ticker1] - beta * precos_teste[ticker2]
                    spread_teste = spread_teste.shift(1)

                    # Concatenar histórico de otimização + teste para inicializar EWM
                    spread_total = pd.concat([df_pares['spread'].iloc[i], spread_teste])

                    # Calcular EWM "aquecida" pelo período de otimização
                    ewm_media_total = spread_total.ewm(halflife=80, adjust=True).mean()
                    ewm_std_total = spread_total.ewm(halflife=80, adjust=True).std()

                    z_score_total = (spread_total - ewm_media_total) / ewm_std_total
                    z_score_movel = z_score_total.iloc[len(df_pares['spread'].iloc[i]):].shift(1)

                    # Selecionar apenas período de teste e aplicar shift(1) para evitar lookahead
                    spread_media_movel_exp = ewm_media_total.iloc[len(df_pares['spread'].iloc[i]):].shift(1)
                    spread_desvio_movel_exp = ewm_std_total.iloc[len(df_pares['spread'].iloc[i]):].shift(1)

                    # Calcular z-score
                    z_score_teste = (spread_teste - spread_media_movel_exp) / spread_desvio_movel_exp

                    
                    # Calcular quantis com base no histórico do spread
                    q_entrada_long = z_score_total.quantile(0.05)
                    q_saida_long   = z_score_total.quantile(0.25)
                    q_entrada_short= z_score_total.quantile(0.95)
                    q_saida_short  = z_score_total.quantile(0.75)

                    q_sl_long  = z_score_total.quantile(0.005)
                    q_tp_long  = z_score_total.quantile(0.50)
                    q_sl_short = z_score_total.quantile(0.995)
                    q_tp_short = z_score_total.quantile(0.50)


                    # Sinais
                    entrada_long = z_score_teste < q_entrada_long
                    saida_long   = (z_score_teste > q_saida_long) | (z_score_teste < q_sl_long) | (z_score_teste > q_tp_long) 

                    entrada_short = z_score_teste > q_entrada_short
                    saida_short   = (z_score_teste < q_saida_short) | (z_score_teste > q_sl_short) | (z_score_teste < q_tp_short)

                    # Stop pot vol
                    sl_stop_vol = 2*df_pares['vol-spread'].iloc[i]
                    tp_stop_vol = 4*df_pares['vol-spread'].iloc[i]

                    # Backtest com vectorbt
                    pf = vbt.Portfolio.from_signals(
                    precos_teste[ticker1],
                    entries=entrada_long,
                    exits=saida_long,
                    short_entries=entrada_short,
                    short_exits=saida_short,
                    fees=0.001,
                    freq='D',
                    init_cash=10_000,
                    sl_stop=sl_stop_vol,
                    tp_stop=tp_stop_vol
                    )

                    print(pf.stats())
                    pf.plot().show()
                    lista_portfolio.append(pf)
                except Exception as e:
                    print(f"Erro no backtest para o par {ticker1}/{ticker2}: {e}")
                    continue

            if lista_portfolio:
                # Equity curves de cada par
                curva_equity = pd.concat([pf.value() for pf in lista_portfolio[-len(df_pares):]], axis=1)

                # Pesos por volatilidade inversa
                volatilidade = np.array([df_pares['vol-spread'].iloc[i] for i in range(len(df_pares))])
                pesos = 1 / volatilidade
                pesos /= pesos.sum()
                pesos = pd.Series(pesos, index=curva_equity.columns)

                # Equity do portfólio ponderado
                portfolio_equity = curva_equity.mul(pesos, axis=1).sum(axis=1)

                # Dados do CDI
                cdi= {
                    "2020": 0.0275,
                    "2021": 0.0440,
                    "2022": 0.1237,
                    "2023": 0.1088,
                    "2024": 0.1088
                }


                # Métricas do portfólio
                portfolio_return = (portfolio_equity[-1] / portfolio_equity[0]) - 1
                ano_teste = str(inicio_teste.year)
                cdi_diario = ((1 + cdi[ano_teste]) ** (1 / 252)) - 1
                retornos_diarios = portfolio_equity.pct_change().dropna()
                portfolio_sharpe = ((retornos_diarios - cdi_diario).mean() / (retornos_diarios.std())) * np.sqrt(252)

                resultados.append({
                    "Período": f"{inicio_teste.date()} a {fim_teste.date()}",
                    "Retorno": portfolio_return,
                    "Sharpe": portfolio_sharpe
                })

            print("Resultados finais:")
            for resultado in resultados:
                print(f"Período: {resultado['Período']} | Retorno: {resultado['Retorno']*100:.2f}% | Sharpe: {resultado['Sharpe']:.2f}")

        except Exception as e:
            print(f"Erro no backtest para o par {ticker1}/{ticker2}: {e}")
            continue

In [65]:
run_walk_forward()


=== Período 1 ===
Otimização: 2017-01-01 a 2019-12-31 (3 anos)
Teste: 2020-01-01 a 2020-12-31 (1 ano)
Start                         2020-01-02 00:00:00
End                           2020-12-30 00:00:00
Period                          248 days 00:00:00
Start Value                               10000.0
End Value                            13633.485195
Total Return [%]                        36.334852
Benchmark Return [%]                   -10.226019
Max Gross Exposure [%]                      100.0
Total Fees Paid                         16.330205
Max Drawdown [%]                         7.814154
Max Drawdown Duration            20 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                          36.371187
Worst Trade [%]                         36.3711

Start                         2020-01-02 00:00:00
End                           2020-12-30 00:00:00
Period                          248 days 00:00:00
Start Value                               10000.0
End Value                            10211.095799
Total Return [%]                         2.110958
Benchmark Return [%]                     2.311387
Max Gross Exposure [%]                      100.0
Total Fees Paid                          29.95007
Max Drawdown [%]                        35.354734
Max Drawdown Duration           149 days 00:00:00
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                       -3787.108187
Win Rate [%]                                100.0
Best Trade [%]                          40.022022
Worst Trade [%]                         40.022022
Avg Winning Trade [%]                   40.022022
Avg Losing Trade [%]                          NaN


Start                         2020-01-02 00:00:00
End                           2020-12-30 00:00:00
Period                          248 days 00:00:00
Start Value                               10000.0
End Value                            13537.629366
Total Return [%]                        35.376294
Benchmark Return [%]                    -5.052344
Max Gross Exposure [%]                      100.0
Total Fees Paid                         16.425965
Max Drawdown [%]                         5.045899
Max Drawdown Duration           199 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                           35.41167
Worst Trade [%]                          35.41167
Avg Winning Trade [%]                    35.41167
Avg Losing Trade [%]                          NaN


Resultados finais:
Período: 2020-01-01 a 2020-12-31 | Retorno: 24.45% | Sharpe: 1.24

=== Período 2 ===
Otimização: 2018-01-01 a 2020-12-31 (3 anos)
Teste: 2021-01-01 a 2021-12-31 (1 ano)
Start                         2021-01-04 00:00:00
End                           2021-12-30 00:00:00
Period                          247 days 00:00:00
Start Value                               10000.0
End Value                            10678.639858
Total Return [%]                         6.786399
Benchmark Return [%]                   -10.355515
Max Gross Exposure [%]                      100.0
Total Fees Paid                         19.282098
Max Drawdown [%]                        10.020955
Max Drawdown Duration           203 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [

Start                         2021-01-04 00:00:00
End                           2021-12-30 00:00:00
Period                          247 days 00:00:00
Start Value                               10000.0
End Value                            12255.251728
Total Return [%]                        22.552517
Benchmark Return [%]                   -26.438447
Max Gross Exposure [%]                      100.0
Total Fees Paid                         17.707061
Max Drawdown [%]                          3.36508
Max Drawdown Duration           170 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                           22.57507
Worst Trade [%]                          22.57507
Avg Winning Trade [%]                    22.57507
Avg Losing Trade [%]                          NaN


...

Start                         2023-01-02 00:00:00
End                           2023-12-28 00:00:00
Period                          248 days 00:00:00
Start Value                               10000.0
End Value                            14200.163009
Total Return [%]                         42.00163
Benchmark Return [%]                    51.479152
Max Gross Exposure [%]                      100.0
Total Fees Paid                         43.794816
Max Drawdown [%]                         8.337976
Max Drawdown Duration            63 days 00:00:00
Total Trades                                    2
Total Closed Trades                             2
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 50.0
Best Trade [%]                          45.015532
Worst Trade [%]                         -2.050002
Avg Winning Trade [%]                   45.015532
Avg Losing Trade [%]                    -2.050002


Resultados finais:
Período: 2020-01-01 a 2020-12-31 | Retorno: 24.45% | Sharpe: 1.24
Período: 2021-01-01 a 2021-12-31 | Retorno: 16.41% | Sharpe: 1.11
Período: 2022-01-01 a 2022-12-31 | Retorno: -0.90% | Sharpe: -1.23
Período: 2023-01-01 a 2023-12-31 | Retorno: 28.97% | Sharpe: 1.69

=== Período 5 ===
Otimização: 2021-01-01 a 2023-12-31 (3 anos)
Teste: 2024-01-01 a 2024-12-31 (1 ano)
Start                         2024-01-02 00:00:00
End                           2024-12-30 00:00:00
Period                          251 days 00:00:00
Start Value                               10000.0
End Value                            10118.151868
Total Return [%]                         1.181519
Benchmark Return [%]                   -56.745042
Max Gross Exposure [%]                      100.0
Total Fees Paid                          20.11829
Max Drawdown [%]                        11.986312
Max Drawdown Duration           209 days 00:00:00
Total Trades                                    1
Total Closed 

Start                         2024-01-02 00:00:00
End                           2024-12-30 00:00:00
Period                          251 days 00:00:00
Start Value                               10000.0
End Value                            10989.818641
Total Return [%]                         9.898186
Benchmark Return [%]                   -19.451035
Max Gross Exposure [%]                      100.0
Total Fees Paid                          18.97123
Max Drawdown [%]                         9.217325
Max Drawdown Duration           177 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                           9.908085
Worst Trade [%]                          9.908085
Avg Winning Trade [%]                    9.908085
Avg Losing Trade [%]                          NaN


Start                         2024-01-02 00:00:00
End                           2024-12-30 00:00:00
Period                          251 days 00:00:00
Start Value                               10000.0
End Value                              9350.49463
Total Return [%]                        -6.495054
Benchmark Return [%]                         -6.0
Max Gross Exposure [%]                      100.0
Total Fees Paid                         60.142436
Max Drawdown [%]                        31.997009
Max Drawdown Duration           220 days 00:00:00
Total Trades                                    3
Total Closed Trades                             3
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            33.333333
Best Trade [%]                           3.207656
Worst Trade [%]                         -5.491635
Avg Winning Trade [%]                    3.207656
Avg Losing Trade [%]                    -4.817526


Resultados finais:
Período: 2020-01-01 a 2020-12-31 | Retorno: 24.45% | Sharpe: 1.24
Período: 2021-01-01 a 2021-12-31 | Retorno: 16.41% | Sharpe: 1.11
Período: 2022-01-01 a 2022-12-31 | Retorno: -0.90% | Sharpe: -1.23
Período: 2023-01-01 a 2023-12-31 | Retorno: 28.97% | Sharpe: 1.69
Período: 2024-01-01 a 2024-12-31 | Retorno: 3.00% | Sharpe: -0.70


# Discussão dos Resultados do Backtest Anual com Comparações

Os resultados finais do backtest da estratégia de **Pairs Trading** são apresentados abaixo, junto com o **CDI anual** e **Ibovespa** para referência:

| Período | Retorno Estratégia (%) | Sharpe | Retorno CDI (%) | Retorno Ibovespa (%) |
|---------|----------------------|--------|----------------|--------------------|
| 2020    | 24.45                | 1.24   | 2.75           | -1.99              |
| 2021    | 16.41                | 1.11   | 2.65           | 2.92               |
| 2022    | -0.90                | -1.23  | 9.90           | 4.08               |
| 2023    | 28.97                | 1.69   | 5.80           | 13.30              |
| 2024    | 3.00                 | -0.70  | 6.20           | 1.50               |

---

## 🔹 Observações

1. **2020 e 2021**
   - Estratégia superou amplamente **CDI** e, em 2020, também superou o **Ibovespa**.
   - Sharpe acima de 1 indica risco-retorno positivo, mostrando que a estratégia capturou bem reversões à média.

2. **2022**
   - Retorno negativo da estratégia (-0.90%) comparado com **CDI (9.9%)** e **Ibovespa (4.08%)**.
   - Mostra que, em anos de alta volatilidade e forte tendência de mercado, a estratégia sofreu, enquanto renda fixa teve desempenho consistente.

3. **2023**
   - Excelente desempenho (28.97%), superando tanto **CDI** quanto **Ibovespa**.
   - Sharpe elevado (1.69) indica eficiência na captura de oportunidades de reversão.

4. **2024**
   - Retorno modesto (3%) com Sharpe negativo (-0.70), inferior a **CDI (6.2%)**, mas ainda acima do **Ibovespa (1.5%)**.
   - Indica que o portfólio enfrentou pouca oportunidade de reversão ou pares perderam cointegridade.

---

## 🔹 Conclusões

- A estratégia demonstra **potencial de alpha**, superando consistentemente o CDI e o Ibovespa em anos favoráveis (2020, 2021, 2023).
- Em anos com forte tendência ou choques de mercado (2022), **renda fixa se mostra mais segura**, evidenciando a importância de gestão de risco e hedge.
- Sharpe negativo em 2022 e 2024 reforça a necessidade de:
  - Reavaliação anual dos pares;
  - Ajuste de beta ou hedge dinâmico;
  - Monitoramento contínuo da cointegridade dos pares e da volatilidade do spread.

---

> 🔹 Em resumo: a estratégia tem **bom desempenho relativo ao mercado e à renda fixa**, mas precisa de **adaptação contínua** para manter robustez em diferentes regimes de mercado.

